In [3]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.examples.tutorials.mnist import input_data

import warnings
warnings.filterwarnings('ignore')

mnist = input_data.read_data_sets('MNIST_data/', one_hot=True)
trainimg = mnist.train.images
trainlabel = mnist.train.labels
testimg = mnist.test.images
testlabel = mnist.test.labels

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [4]:
n_input = 784
n_output = 10
stddev = 0.1
weights = {
    'wc1': tf.Variable(tf.random_normal([3, 3, 1, 64],stddev=stddev)),
    'wc2': tf.Variable(tf.random_normal([3, 3, 64, 128],stddev=stddev)),
    'wd1': tf.Variable(tf.random_normal([7*7*128, 1024],stddev=stddev)),
    'wd2': tf.Variable(tf.random_normal([1024, n_output],stddev=stddev)),
}
biases = {
    'bc1': tf.Variable(tf.random_normal([64],stddev=stddev)),
    'bc2': tf.Variable(tf.random_normal([128],stddev=stddev)),
    'bd1': tf.Variable(tf.random_normal([1024],stddev=stddev)),
    'bd2': tf.Variable(tf.random_normal([n_output],stddev=stddev))
}

In [5]:
def cov_basic(_input, _w, _b, _keepratio):
    # INPUT ->(n,h,w,c) 
    # filter -> (f_h, f_w, filter in_channels, filter out_channels)
    _input_r = tf.reshape(_input,shape=[-1, 28, 28, 1])
    # CONV LAYER 1
    # strides: first 1 represents stride of batch_size;second 1 represents 
    #          stride of h;third 1 represents stride of w;four 1 represents stride of c
    _conv1 = tf.nn.conv2d(_input_r, _w['wc1'], strides=[1, 1, 1, 1], padding='SAME')
    # _mean, _var = tf.nn.moments(_conv1, [0, 1, 2])
    #_conv1 = tf.nn.batch_normalization(_conv1, _mean, _var, 0, 1, 0.0001)
    _conv1 = tf.nn.relu(tf.nn.bias_add(_conv1, _b['bc1']))
    # ksize ->[batch_size, h , w, c]
    _pool1 = tf.nn.max_pool(_conv1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
    _pool_dr1 = tf.nn.dropout(_pool1, _keepratio)
    # CONV LAYER 2
    _conv2 = tf.nn.conv2d(_pool_dr1, _w['wc2'], strides=[1, 1, 1, 1], padding='SAME')
    # _mean, _var = tf.nn.moments(_conv2, [0, 1, 2])
    #_conv2 = tf.nn.batch_normalization(_conv2, _mean, _var, 0, 1, 0.0001)
    _conv2 = tf.nn.relu(tf.nn.bias_add(_conv2, _b['bc2']))
    _pool2 = tf.nn.max_pool(_conv2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
    _pool_dr2 = tf.nn.dropout(_pool2, _keepratio)
    # VECTORIZE
    _densel = tf.reshape(_pool_dr2, [-1, _w['wd1'].get_shape().as_list()[0]])
    # FULLY CONNECTED LAYER 1
    _fc1 = tf.nn.relu(tf.add(tf.matmul(_densel, _w['wd1']), _b['bd1']))
    _fc_dr1 = tf.nn.dropout(_fc1, _keepratio)
    # FULLY CONNECTED LAYER 2
    _out = tf.add(tf.matmul(_fc_dr1, _w['wd2']), _b['bd2'])
    # RETURN
    out = { 'input_r':_input_r, 'conv1':_conv1, 'pool1':_pool1, 'pool_dr1':_pool_dr1,
            'conv2':_conv2, 'pool2':_pool2, 'pool_dr2':_pool_dr2, 'densel':_densel,
            'fc1':_fc1, 'fc_dr1':_fc_dr1, 'out':_out
          }
    return out

In [7]:
a = tf.Variable(tf.random_normal([3, 3, 1, 64], stddev=0.1))
print(a)
a = tf.Print(a,[a],'a: ')
b =  weights['wd1'].get_shape().as_list()[0]
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)
print(b)

<tf.Variable 'Variable_9:0' shape=(3, 3, 1, 64) dtype=float32_ref>
6272


In [5]:
# print(help(tf.nn.conv2d))
# print(help(tf.nn.max_pool))

In [9]:
x = tf.placeholder("float", [None, n_input])
y = tf.placeholder('float', [None, n_output])
keepratio = tf.placeholder(tf.float32)

# Functions

_pred = cov_basic(x, weights, biases, keepratio)['out']
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=_pred, labels=y))
optm = tf.train.AdamOptimizer(learning_rate=0.001).minimize(cost)
_corr = tf.equal(tf.arg_max(_pred,1),tf.arg_max(y,1))
accr = tf.reduce_mean(tf.cast(_corr,'float'))
init = tf.global_variables_initializer()

# SAVER
save_step = 1
saver = tf.train.Saver(max_to_keep=3)

In [12]:
do_train = 0

In [13]:
training_epochs = 15
batch_size = 16
display_step = 1
#Session
sess = tf.Session()
sess.run(init)
# mini-batch learning
if do_train == 1:
    for epoch in range(training_epochs):
        avg_cost = 0
        # total_batch = int(mnist.train.num_examples/batch_size)
        total_batch=10
        for i in range(total_batch):
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)
            feeds={x:batch_xs, y:batch_ys, keepratio:0.7}
            sess.run(optm,feed_dict=feeds)
            avg_cost += sess.run(cost, feed_dict={x:batch_xs, y:batch_ys, keepratio:1.0})/total_batch
        #display
        if epoch % display_step == 0:
            feeds_train = {x:batch_xs, y:batch_ys, keepratio:1.0}
    #         feeds_test = {x:mnist.test.images, y:mnist.test.labels, keepratio:1.0}
            train_acc = sess.run(accr, feed_dict=feeds_train)
    #         test_acc = sess.run(accr, feed_dict = feeds_test)
            print("Epoch: %03d/%03d cost: %.9f train_acc: %.3f "%
                 (epoch, training_epochs, avg_cost, train_acc))

        if epoch % save_step == 0:
            saver.save(sess,'save/nets/cnn_mnist_basic.ckpt-'+str(epoch))
print('Done')

Done


In [15]:
if do_train ==0:
    epoch = training_epochs-1
    saver.restore(sess,'save/nets/cnn_mnist_basic.ckpt-'+str(epoch))
    for i in range(20):
        batch_test_xs, batch_test_ys = mnist.test.next_batch(500)
        test_acc = sess.run(accr, feed_dict = {x: batch_test_xs, y:batch_test_ys, keepratio:1.})
    print('Test Accuracy : %.3f'%(test_acc))

INFO:tensorflow:Restoring parameters from save/nets/cnn_mnist_basic.ckpt-14
Test Accuracy : 0.844
